In [1]:
# モジュールのインポート
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.layers.merge import concatenate
from keras.models import Model

# 制度評価用モジュールのインポート
from keras.optimizers import Adam
import keras.backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.
C:\Users\r07-n\AppData\Roaming\Python\Python37\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\r07-n\AppData\Roaming\Python\Python37\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\r07-n\AppData\Roaming\Python\Python37\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Us

In [2]:
# ディレクトリ内の画像を読み込む
# inputpath: ディレクトリ文字列，imagesize: 画像サイズ，type_color: ColorかGray
def load_images(inputpath, imagesize, type_color):
    imglist = []
    exclude_prefixes = ('__', '.')
    
    for root, dirs, files in os.walk(inputpath):
        for fn in sorted(files):
            dirs[:] = [dir for dir in dirs if not dir.startswith(exclude_prefixes)]
            files[:] = [file for file in files if not file.startswith(exclude_prefixes)]
            
        for fn in sorted(files):
            bn, ext = os.path.splitext(fn)
            if ext not in [".bmp", ".BMP", ".jpg", ".JPG", ".jpeg", ".JPEG", ".png", ".PNG"]:
                continue

            filename = os.path.join(root, fn)
            
            if type_color == 'Color':
                # カラー画像の場合
                testimage = cv2.imread(filename, cv2.IMREAD_COLOR)
                # サイズ変更
                height, width = testimage.shape[:2]
                testimage = cv2.resize(testimage, (imagesize, imagesize), interpolation = cv2.INTER_AREA)  #主に縮小するのでINTER_AREA使用
                testimage = np.asarray(testimage, dtype=np.float64)
                # チャンネル，高さ，幅に入れ替え．data_format="channels_first"を使うとき必要
                #testimage = testimage.transpose(2, 0, 1)
                # チャンネルをbgrの順からrgbの順に変更
                testimage = testimage[::-1]
            
            elif type_color == 'Gray':
                # グレースケール画像の場合
                testimage = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
                # サイズ変更
                height, width = testimage.shape[:2]
                testimage = cv2.resize(testimage, (imagesize, imagesize), interpolation = cv2.INTER_AREA)  #主に縮小するのでINTER_AREA使用
                # チャンネルの次元がないので1次元追加する
                testimage = np.asarray([testimage], dtype=np.float64)
                testimage = np.asarray(testimage, dtype=np.float64).reshape((1, imagesize, imagesize))
                # 高さ，幅，チャンネルに入れ替え．data_format="channels_last"を使うとき必要
                testimage = testimage.transpose(1, 2, 0)

            imglist.append(testimage)
    imgsdata = np.asarray(imglist, dtype=np.float32)

    return imgsdata # 画像リストとファイル名のリストを返す

In [3]:
# 指定ディレクトリに画像リストの画像を保存する
# savepath: ディレクトリ文字列, filenamelist: ファイル名リスト, imagelist: 画像リスト
def save_images(savepath, filenamelist, imagelist):
    for i, fn in enumerate(filenamelist):
        filename = os.path.join(savepath, fn)
        testimage = imagelist[i]
        testimage = np.delete(testimage, 2, 1) # グレースケール画像を保存するときだけ使用．チャンネルに相当する3列目削除
        cv2.imwrite(filename, testimage)

In [4]:
# 目的のラベル番号: orgnum のみを outnum にして，他はゼロにする関数
def convlabelnum2desire(label, orgnum, outnum, w, h):
    for case in range(0, len(label)):
        for i in range(0, h):
            for j in range(0, w):
                if(label[case][i][j][0] != orgnum):
                    label[case][i][j][0] = 0.0
                else:
                    label[case][i][j][0] = outnum

In [5]:
IMAGE_SIZE = 256
SEGMENTATION_TARGET_LABEL = 255
EPOCHS = 20

In [6]:
# データ準備
# 画像読み込み
# training用の原画像とラベル画像読み込み(2116枚)
image_train = load_images(r'C:\medical_image\x_ray_images\train original 50', IMAGE_SIZE, 'Gray')
label_train = load_images(r'C:\medical_image\x_ray_images\train label 50', IMAGE_SIZE, 'Gray')

# test用の原画像とラベル画像読み込み(553枚)
image_test  = load_images(r'C:\medical_image\x_ray_images\test original 15', IMAGE_SIZE, 'Gray')
label_test  = load_images(r'C:\medical_image\x_ray_images\test label 15', IMAGE_SIZE, 'Gray')

# ラベル画像は指定した値 SEGMENTATION_TARGET_LABEL だけ残し他は0にする
convlabelnum2desire(label_train, SEGMENTATION_TARGET_LABEL, 255, IMAGE_SIZE, IMAGE_SIZE)
convlabelnum2desire(label_test, SEGMENTATION_TARGET_LABEL, 255, IMAGE_SIZE, IMAGE_SIZE)

# 画素値0-1正規化
image_train /= np.max(image_train)
label_train /= np.max(label_train)
image_test /= np.max(image_test)
label_test /= np.max(label_test)

In [7]:
# 評価指標
def true_positive(y_true, y_pred):
    return K.sum(K.cast(K.equal(y_true * y_pred, 1), K.floatx()))

def true_negative(y_true, y_pred):
    return K.sum(K.cast(K.equal(y_true + y_pred, 0), K.floatx()))

def false_positive(y_true, y_pred):
    return K.sum(K.cast(K.less(y_true, y_pred), K.floatx()))

def false_negative(y_true, y_pred):
    return K.sum(K.cast(K.greater(y_true, y_pred), K.floatx()))
#DICE係数を計算する関数
def dice_coef(y_true, y_pred, smooth=1e-9):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

#ロス関数
def dice_coef_loss(y_true, y_pred):
    return (1.0 - dice_coef(y_true, y_pred))

# IoU（評価関数)
def iou_score(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return (true_positive(y_true, y_pred) / (false_negative(y_true, y_pred)+true_positive(y_true, y_pred)+false_positive(y_true, y_pred)))

In [8]:
# U-net
# IMAGE_SIZE = 256
def network_unet():
    input_img = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1))
    
    enc1 = Conv2D(64, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(input_img)
    enc1 = Conv2D(64, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(enc1)
    enc1 = BatchNormalization()(enc1)
    down1 = MaxPooling2D(pool_size=2, strides=2)(enc1)
    
    enc2 = Conv2D(128, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(down1)
    enc2 = Conv2D(128, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(enc2)
    enc2 = BatchNormalization()(enc2)
    down2 = MaxPooling2D(pool_size=2, strides=2)(enc2)
    
    enc3 = Conv2D(256, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(down2)
    enc3 = Conv2D(256, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(enc3)
    enc3 = BatchNormalization()(enc3)
    down3 = MaxPooling2D(pool_size=2, strides=2)(enc3)
    
    enc4 = Conv2D(512, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(down3)
    enc4 = Conv2D(512, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(enc4)
    enc4 = BatchNormalization()(enc4)
    down4 = MaxPooling2D(pool_size=2, strides=2)(enc4)
    
    enc5 = Conv2D(1024, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(down4)
    enc5 = Conv2D(1024, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(enc5)
    enc5 = BatchNormalization()(enc5)
    
    up4 = Conv2D(512, kernel_size=2, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(UpSampling2D(size=(2, 2))(enc5))
    merge4 = concatenate([enc4, up4], axis=3)
    dec4 = Conv2D(512, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(merge4)
    dec4 = Conv2D(512, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(dec4)
    dec4 = BatchNormalization()(dec4)
    
    up3 = Conv2D(256, kernel_size=2, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(UpSampling2D(size=(2, 2))(dec4))
    merge3 = concatenate([enc3, up3], axis=3)
    dec3 = Conv2D(256, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(merge3)
    dec3 = Conv2D(256, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(dec3)
    dec3 = BatchNormalization()(dec3)
    
    up2 = Conv2D(128, kernel_size=2, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(UpSampling2D(size=(2, 2))(dec3))
    merge2 = concatenate([enc2, up2], axis=3)
    dec2 = Conv2D(128, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(merge2)
    dec2 = Conv2D(128, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(dec2)
    dec2 = BatchNormalization()(dec2)
    
    up1 = Conv2D(64, kernel_size=2, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(UpSampling2D(size=(2, 2))(dec2))
    merge1 = concatenate([enc1, up1], axis=3)
    dec1 = Conv2D(64, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(merge1)
    dec1 = Conv2D(64, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(dec1)
    dec1 = BatchNormalization()(dec1)
    dec1 = Conv2D(2, kernel_size=3, strides=1, activation="relu", padding="same", kernel_initializer="he_normal")(dec1)
    dec1 = Conv2D(1, kernel_size=1, strides=1, activation="sigmoid", padding="same")(dec1)
    
    model = Model(inputs=input_img, outputs=dec1)
    
    return model

model = network_unet()

# ネットワークを表示
print(model.summary())    

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_2[0][0]                   
____________________________________________________________________________________________

In [9]:
# U-Netモデルのコンパイル
model.compile(optimizer="Adam", loss=dice_coef_loss, metrics=[dice_coef])

In [10]:
# モデルの学習が進まなくなった場合、自動的に学習を終わらせる
early_stop = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduceLROnPlat = ReduceLROnPlateau(monitor="val_loss", factor=0.25, min_delta=1e-10, patience=3,
                                  verbose=1, mode="auto")

callbacks = [reduceLROnPlat]

In [12]:
# training実行
# training用原画像とラベル画像でtrainingする．validation用にtest用の原画像とラベル画像を使用する．
training = model.fit(image_train, label_train, epochs=70, batch_size=16, shuffle=True, 
                    validation_data=(image_test, label_test), verbose=1, callbacks=callbacks)

Train on 50 samples, validate on 15 samples
Epoch 1/70
50/50 [==============================] - 470s 9s/step - loss: 0.0417 - dice_coef: 0.9595 - val_loss: 0.4974 - val_dice_coef: 0.5026
Epoch 2/70
50/50 [==============================] - 468s 9s/step - loss: 0.0420 - dice_coef: 0.9576 - val_loss: 0.4812 - val_dice_coef: 0.5188
Epoch 3/70
50/50 [==============================] - 443s 9s/step - loss: 0.0406 - dice_coef: 0.9474 - val_loss: 0.4795 - val_dice_coef: 0.5205
Epoch 4/70
50/50 [==============================] - 369s 7s/step - loss: 0.0382 - dice_coef: 0.9639 - val_loss: 0.4735 - val_dice_coef: 0.5265
Epoch 5/70
50/50 [==============================] - 367s 7s/step - loss: 0.0363 - dice_coef: 0.9636 - val_loss: 0.4477 - val_dice_coef: 0.5523
Epoch 6/70
50/50 [==============================] - 359s 7s/step - loss: 0.0370 - dice_coef: 0.9630 - val_loss: 0.4522 - val_dice_coef: 0.5478
Epoch 7/70
50/50 [==============================] - 353s 7s/step - loss: 0.0348 - dice_coef: 0.963

Epoch 56/70
50/50 [==============================] - 367s 7s/step - loss: 0.0315 - dice_coef: 0.9688 - val_loss: 0.3761 - val_dice_coef: 0.6239
Epoch 57/70
50/50 [==============================] - 372s 7s/step - loss: 0.0311 - dice_coef: 0.9674 - val_loss: 0.3744 - val_dice_coef: 0.6256
Epoch 58/70
50/50 [==============================] - 374s 7s/step - loss: 0.0307 - dice_coef: 0.9686 - val_loss: 0.3731 - val_dice_coef: 0.6269
Epoch 59/70
50/50 [==============================] - 371s 7s/step - loss: 0.0310 - dice_coef: 0.9699 - val_loss: 0.3715 - val_dice_coef: 0.6285
Epoch 60/70
50/50 [==============================] - 374s 7s/step - loss: 0.0310 - dice_coef: 0.9695 - val_loss: 0.3696 - val_dice_coef: 0.6304
Epoch 61/70
50/50 [==============================] - 374s 7s/step - loss: 0.0308 - dice_coef: 0.9703 - val_loss: 0.3676 - val_dice_coef: 0.6324
Epoch 62/70
50/50 [==============================] - 378s 8s/step - loss: 0.0308 - dice_coef: 0.9695 - val_loss: 0.3660 - val_dice_coef:

In [13]:
# training実行
# training用原画像とラベル画像でtrainingする．validation用にtest用の原画像とラベル画像を使用する．
training = model.fit(image_train, label_train, epochs=30, batch_size=16, shuffle=True, 
                    validation_data=(image_test, label_test), verbose=1, callbacks=callbacks)

Train on 50 samples, validate on 15 samples
Epoch 1/30
50/50 [==============================] - 504s 10s/step - loss: 0.0307 - dice_coef: 0.9695 - val_loss: 0.3493 - val_dice_coef: 0.6507
Epoch 2/30
50/50 [==============================] - 545s 11s/step - loss: 0.0310 - dice_coef: 0.9696 - val_loss: 0.3472 - val_dice_coef: 0.6528
Epoch 3/30
50/50 [==============================] - 518s 10s/step - loss: 0.0312 - dice_coef: 0.9674 - val_loss: 0.3450 - val_dice_coef: 0.6550
Epoch 4/30
50/50 [==============================] - 478s 10s/step - loss: 0.0306 - dice_coef: 0.9660 - val_loss: 0.3423 - val_dice_coef: 0.6577
Epoch 5/30
50/50 [==============================] - 479s 10s/step - loss: 0.0315 - dice_coef: 0.9665 - val_loss: 0.3399 - val_dice_coef: 0.6601
Epoch 6/30
50/50 [==============================] - 480s 10s/step - loss: 0.0317 - dice_coef: 0.9674 - val_loss: 0.3364 - val_dice_coef: 0.6636
Epoch 7/30
50/50 [==============================] - 481s 10s/step - loss: 0.0310 - dice_coef

In [14]:
# training実行
# training用原画像とラベル画像でtrainingする．validation用にtest用の原画像とラベル画像を使用する．
training = model.fit(image_train, label_train, epochs=30, batch_size=16, shuffle=True, 
                    validation_data=(image_test, label_test), verbose=1, callbacks=callbacks)

Train on 50 samples, validate on 15 samples
Epoch 1/30
50/50 [==============================] - 380s 8s/step - loss: 0.0304 - dice_coef: 0.9683 - val_loss: 0.2713 - val_dice_coef: 0.7287
Epoch 2/30
50/50 [==============================] - 387s 8s/step - loss: 0.0314 - dice_coef: 0.9679 - val_loss: 0.2680 - val_dice_coef: 0.7320
Epoch 3/30
50/50 [==============================] - 383s 8s/step - loss: 0.0299 - dice_coef: 0.9686 - val_loss: 0.2657 - val_dice_coef: 0.7343
Epoch 4/30
50/50 [==============================] - 388s 8s/step - loss: 0.0308 - dice_coef: 0.9677 - val_loss: 0.2641 - val_dice_coef: 0.7359
Epoch 5/30
50/50 [==============================] - 391s 8s/step - loss: 0.0305 - dice_coef: 0.9700 - val_loss: 0.2630 - val_dice_coef: 0.7370
Epoch 6/30
50/50 [==============================] - 391s 8s/step - loss: 0.0304 - dice_coef: 0.9703 - val_loss: 0.2624 - val_dice_coef: 0.7376
Epoch 7/30
50/50 [==============================] - 388s 8s/step - loss: 0.0307 - dice_coef: 0.970

In [15]:
# training実行
# training用原画像とラベル画像でtrainingする．validation用にtest用の原画像とラベル画像を使用する．
training = model.fit(image_train, label_train, epochs=30, batch_size=16, shuffle=True, 
                    validation_data=(image_test, label_test), verbose=1, callbacks=callbacks)

Train on 50 samples, validate on 15 samples
Epoch 1/30
50/50 [==============================] - 436s 9s/step - loss: 0.0303 - dice_coef: 0.9662 - val_loss: 0.2171 - val_dice_coef: 0.7829
Epoch 2/30
50/50 [==============================] - 410s 8s/step - loss: 0.0302 - dice_coef: 0.9661 - val_loss: 0.2136 - val_dice_coef: 0.7864
Epoch 3/30
50/50 [==============================] - 434s 9s/step - loss: 0.0297 - dice_coef: 0.9708 - val_loss: 0.2121 - val_dice_coef: 0.7879
Epoch 4/30
50/50 [==============================] - 429s 9s/step - loss: 0.0300 - dice_coef: 0.9698 - val_loss: 0.2112 - val_dice_coef: 0.7888
Epoch 5/30
50/50 [==============================] - 441s 9s/step - loss: 0.0300 - dice_coef: 0.9661 - val_loss: 0.2116 - val_dice_coef: 0.7884
Epoch 6/30
50/50 [==============================] - 459s 9s/step - loss: 0.0298 - dice_coef: 0.9694 - val_loss: 0.2094 - val_dice_coef: 0.7906
Epoch 7/30
50/50 [==============================] - 456s 9s/step - loss: 0.0308 - dice_coef: 0.968

In [16]:
# training実行
# training用原画像とラベル画像でtrainingする．validation用にtest用の原画像とラベル画像を使用する．
training = model.fit(image_train, label_train, epochs=60, batch_size=16, shuffle=True, 
                    validation_data=(image_test, label_test), verbose=1, callbacks=callbacks)

Train on 50 samples, validate on 15 samples
Epoch 1/60
50/50 [==============================] - 370s 7s/step - loss: 0.0290 - dice_coef: 0.9695 - val_loss: 0.1700 - val_dice_coef: 0.8300
Epoch 2/60
50/50 [==============================] - 371s 7s/step - loss: 0.0288 - dice_coef: 0.9710 - val_loss: 0.1695 - val_dice_coef: 0.8305
Epoch 3/60
50/50 [==============================] - 381s 8s/step - loss: 0.0290 - dice_coef: 0.9701 - val_loss: 0.1685 - val_dice_coef: 0.8315
Epoch 4/60
50/50 [==============================] - 400s 8s/step - loss: 0.0296 - dice_coef: 0.9679 - val_loss: 0.1648 - val_dice_coef: 0.8352
Epoch 5/60
50/50 [==============================] - 409s 8s/step - loss: 0.0294 - dice_coef: 0.9714 - val_loss: 0.1622 - val_dice_coef: 0.8378
Epoch 6/60
50/50 [==============================] - 409s 8s/step - loss: 0.0298 - dice_coef: 0.9705 - val_loss: 0.1604 - val_dice_coef: 0.8396
Epoch 7/60
50/50 [==============================] - 415s 8s/step - loss: 0.0291 - dice_coef: 0.969

50/50 [==============================] - 372s 7s/step - loss: 0.0299 - dice_coef: 0.9627 - val_loss: 0.1513 - val_dice_coef: 0.8487
Epoch 53/60
50/50 [==============================] - 377s 8s/step - loss: 0.0297 - dice_coef: 0.9670 - val_loss: 0.1483 - val_dice_coef: 0.8517

Epoch 00053: ReduceLROnPlateau reducing learning rate to 9.313226188509272e-13.
Epoch 54/60
50/50 [==============================] - 378s 8s/step - loss: 0.0296 - dice_coef: 0.9701 - val_loss: 0.1480 - val_dice_coef: 0.8520
Epoch 55/60
50/50 [==============================] - 371s 7s/step - loss: 0.0301 - dice_coef: 0.9661 - val_loss: 0.1461 - val_dice_coef: 0.8539
Epoch 56/60
50/50 [==============================] - 365s 7s/step - loss: 0.0289 - dice_coef: 0.9708 - val_loss: 0.1470 - val_dice_coef: 0.8530
Epoch 57/60
50/50 [==============================] - 361s 7s/step - loss: 0.0287 - dice_coef: 0.9728 - val_loss: 0.1473 - val_dice_coef: 0.8527
Epoch 58/60
50/50 [==============================] - 370s 7s/step -

In [17]:
# training実行
# training用原画像とラベル画像でtrainingする．validation用にtest用の原画像とラベル画像を使用する．
training = model.fit(image_train, label_train, epochs=60, batch_size=8, shuffle=True, 
                    validation_data=(image_test, label_test), verbose=1, callbacks=callbacks)

Train on 50 samples, validate on 15 samples
Epoch 1/60
50/50 [==============================] - 309s 6s/step - loss: 0.0299 - dice_coef: 0.9692 - val_loss: 0.1511 - val_dice_coef: 0.8411
Epoch 2/60
50/50 [==============================] - 319s 6s/step - loss: 0.0307 - dice_coef: 0.9692 - val_loss: 0.1511 - val_dice_coef: 0.8411
Epoch 3/60
50/50 [==============================] - 310s 6s/step - loss: 0.0307 - dice_coef: 0.9678 - val_loss: 0.1498 - val_dice_coef: 0.8425
Epoch 4/60
50/50 [==============================] - 309s 6s/step - loss: 0.0297 - dice_coef: 0.9713 - val_loss: 0.1496 - val_dice_coef: 0.8427
Epoch 5/60
50/50 [==============================] - 307s 6s/step - loss: 0.0307 - dice_coef: 0.9694 - val_loss: 0.1493 - val_dice_coef: 0.8431
Epoch 6/60
50/50 [==============================] - 315s 6s/step - loss: 0.0301 - dice_coef: 0.9698 - val_loss: 0.1476 - val_dice_coef: 0.8449
Epoch 7/60
50/50 [==============================] - 313s 6s/step - loss: 0.0321 - dice_coef: 0.965

50/50 [==============================] - 277s 6s/step - loss: 0.0309 - dice_coef: 0.9654 - val_loss: 0.1397 - val_dice_coef: 0.8534
Epoch 52/60
50/50 [==============================] - 282s 6s/step - loss: 0.0303 - dice_coef: 0.9684 - val_loss: 0.1390 - val_dice_coef: 0.8541
Epoch 53/60
50/50 [==============================] - 277s 6s/step - loss: 0.0300 - dice_coef: 0.9704 - val_loss: 0.1402 - val_dice_coef: 0.8529
Epoch 54/60
50/50 [==============================] - 280s 6s/step - loss: 0.0299 - dice_coef: 0.9696 - val_loss: 0.1406 - val_dice_coef: 0.8524
Epoch 55/60
50/50 [==============================] - 278s 6s/step - loss: 0.0298 - dice_coef: 0.9693 - val_loss: 0.1405 - val_dice_coef: 0.8525

Epoch 00055: ReduceLROnPlateau reducing learning rate to 3.469447116743502e-21.
Epoch 56/60
50/50 [==============================] - 280s 6s/step - loss: 0.0296 - dice_coef: 0.9705 - val_loss: 0.1409 - val_dice_coef: 0.8521
Epoch 57/60
50/50 [==============================] - 279s 6s/step -